In [1]:
import os
import zipfile
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive


def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def parse_xml_file(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
    except UnicodeDecodeError:
        try:
            with open(filepath, 'r', encoding='latin-1') as file:
                content = file.read()
        except UnicodeDecodeError:
            with open(filepath, 'r', encoding='iso-8859-1', errors='ignore') as file:
                content = file.read()

    soup = BeautifulSoup(content, 'xml')

    case_data = {}
    case_data['name'] = soup.find('name').text if soup.find('name') else None
    case_data['AustLII'] = soup.find('AustLII').text if soup.find('AustLII') else None

    catchphrases = soup.find_all('catchphrase')
    case_data['catchphrases'] = [catchphrase.text for catchphrase in catchphrases]

    sentences = soup.find_all('sentence')
    case_data['sentences'] = [sentence.text for sentence in sentences]

    return case_data

def parse_all_xml_files(directory):
    all_cases = []
    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            filepath = os.path.join(directory, filename)
            case_data = parse_xml_file(filepath)
            all_cases.append(case_data)

    return all_cases

# Specify the path to the zip file and the directory to extract to
zip_path = '/content/drive/MyDrive/Legal Analysis.zip'
extract_to = '/content/legal_analysis'

# Extract the zip file
extract_zip(zip_path, extract_to)

# Parse all XML files from the extracted directory
all_cases = parse_all_xml_files(extract_to)

# Convert the list of dictionaries to a pandas DataFrame for easier analysis
df = pd.DataFrame(all_cases)

# Display the DataFrame
print(df)

                                                   name  \
0     Gordon v Commonwealth of Australia [2008] FCA ...   
1     Mahmoud v The Owners* Corporation Strata Plan ...   
2     MZPAO v Minister for Immigration and Citizensh...   
3     Citrus Queensland Pty Ltd v Sunstate Orchards ...   
4     Secretary, Department of Employment and Workpl...   
...                                                 ...   
4187  Walters v Commissioner of Taxation (No. 2) [20...   
4188  Different Solutions Pty Limited v Commissioner...   
4189  Yalniz v Minister for Immigration and Citizens...   
4190  Bradken Resources Pty Ltd v Lynx Engineering C...   
4191  University of Western Australia v Gray (No 4) ...   

                                                AustLII  \
0     http://www.austlii.edu.au/au/cases/cth/FCA/200...   
1     http://www.austlii.edu.au/au/cases/cth/FCA/200...   
2     http://www.austlii.edu.au/au/cases/cth/FCA/200...   
3     http://www.austlii.edu.au/au/cases/cth/FCA/200...

In [4]:
import os
import zipfile
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset

# Path to the ZIP file and extraction
zip_file_path = '/content/drive/MyDrive/Legal Analysis.zip'
extract_to = '/content/legal_analysis'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Function to parse XML files
from bs4 import BeautifulSoup

def parse_xml_file(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
    except UnicodeDecodeError:
        with open(filepath, 'r', encoding='ISO-8859-1') as file:
            content = file.read()

    soup = BeautifulSoup(content, 'xml')

    case_data = {}
    case_data['name'] = soup.find('name').text if soup.find('name') else None
    case_data['AustLII'] = soup.find('AustLII').text if soup.find('AustLII') else None

    catchphrases = soup.find_all('catchphrase')
    case_data['catchphrases'] = [catchphrase.text for catchphrase in catchphrases]

    sentences = soup.find_all('sentence')
    case_data['sentences'] = [sentence.text for sentence in sentences]

    return case_data

def parse_all_xml_files(directory):
    all_cases = []
    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            filepath = os.path.join(directory, filename)
            case_data = parse_xml_file(filepath)
            all_cases.append(case_data)

    return all_cases

# Parse all XML files from the extracted directory
all_cases = parse_all_xml_files(extract_to)

# Convert the list of dictionaries to a pandas DataFrame for easier analysis
df = pd.DataFrame(all_cases)

# Preprocess Data
df['label'] = df['catchphrases'].apply(lambda x: len(x) % 2)  # Simplified binary labels for demonstration
df['text'] = df['sentences'].apply(lambda x: ' '.join(x))

# Reduce the dataset size for faster training
df = df.sample(n=500, random_state=42)

# Split data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Tokenize the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class LegalDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = LegalDataset(train_texts.tolist(), train_labels.tolist(), tokenizer, max_len=128)
test_dataset = LegalDataset(test_texts.tolist(), test_labels.tolist(), tokenizer, max_len=128)

# Define and train the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Binary classification

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Reduced for faster training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=1)
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average='weighted')
    rec = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Evaluate model
results = trainer.evaluate()
print(results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Step,Training Loss
10,0.723600
20,0.693100


{'eval_loss': 0.6599414348602295, 'eval_accuracy': 0.58, 'eval_precision': 0.6157388316151202, 'eval_recall': 0.58, 'eval_f1': 0.45193675889328055, 'eval_runtime': 65.453, 'eval_samples_per_second': 1.528, 'eval_steps_per_second': 0.107, 'epoch': 1.0}
